In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()

In [2]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()


headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

def get_proxie():
    return(
        {
            "http": proxiesDf.sample(1).loc[:,'IP Address'].values[0]
        }    
    )

In [3]:
rawVideos = pd.read_pickle('../datasets/rawVideos.pkl')
print(len(rawVideos))
rawVideos.head(3)

96728


,videoId,titleRaw,shortDesc,publishedTimeRaw,viewCountRaw,lengthRaw
-eLgCDFKD7s,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,Nikit talks about wildlife and his work within...,7 months ago,546 views,16:46
nqSXwWUwpRM,nqSXwWUwpRM,"Puppets, guardian angels of the culture | Pou...",Persian puppets have a reach history and backg...,7 months ago,407 views,18:58
Uq1cdT4MBDc,Uq1cdT4MBDc,Unroll your role | Negar Javaherian | TEDxSUT,Some of us believe that our roles are carved i...,7 months ago,"1,077 views",19:13


In [4]:
# videoDataset = pd.DataFrame(columns=['videoId', 'likes', 'dislikes', 'lengthSeconds', 'keywords', 'views', 'talkText', 'videoDescription', 'currentDate'])

videoDataset = pd.read_pickle('../datasets/rawVideosContent.pkl')
print('Videos scrapeados:', len(videoDataset))
videoDataset.head(3)

Videos scrapeados: 30910


,videoId,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,publishDate
qNvKWhzdpW4,qNvKWhzdpW4,0,0,1001,"[TEDxTalks, English, Life, Anthropology, Compl...",222,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Dr. Kyle William Bishop is a Cedar City native...,1.57697e+09,2019-05-15
huOxk-UhOz0,huOxk-UhOz0,468,4,1161,"[TEDxTalks, English, United States, Life, Fail...",26243,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","With an honest and vulnerable approach, Joey H...",1.57697e+09,2016-05-17
J0_6b-AJdWU,J0_6b-AJdWU,21,1,764,"[TEDxTalks, English, Health, Communication, Cu...",1085,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",How limited and limiting do our lives become w...,1.57697e+09,2018-05-22


In [5]:
targets = rawVideos[~rawVideos['videoId'].isin(videoDataset['videoId'])]['videoId']
print('Pendientes:', len(targets))

Pendientes: 65513


In [6]:
headersYouTube = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'X-YouTube-Client-Name': '1',
    'X-YouTube-Client-Version': '2.20190509',
}


import urllib.parse as urlparse


def getVideoData(video_id):
    s = requests.Session()
    proxies = get_proxie()
    videoUrl = 'https://www.youtube.com/watch?v='+video_id+'&pbj=1'
    response = s.get(videoUrl, headers=headersYouTube, proxies=proxies)
    data = json.loads(response.text)
    
    videoData = data[3]['playerResponse']['videoDetails']
    category =  data[3]['response']['contents']['twoColumnWatchNextResults']['results']['results']['contents']\
                [1]['videoSecondaryInfoRenderer']['metadataRowContainer']['metadataRowContainerRenderer']['rows'][0]\
                ['metadataRowRenderer']['contents'][0]['runs'][0]['text']
    likeData = data[3]['response']['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]\
                ['videoPrimaryInfoRenderer']['videoActions']['menuRenderer']['topLevelButtons']
    
    likes = likeData[0]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label'].\
            replace(' likes', '').replace(' like', '').replace(',', '').replace('No', '0')
    dislikes = likeData[1]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label'].\
            replace(' dislikes', '').replace(' dislike', '').replace(',', '').replace('No', '0')
    
    likes = int(likes)
    dislikes = int(dislikes)
    
    seconds = videoData['lengthSeconds']
    talkText = getSub(data)
    videoDescription = data[3]['playerResponse']['videoDetails']['shortDescription']
    keywords = videoData['keywords']
    views = videoData['viewCount']
    
    publishDate =  data[3]['playerResponse']['microformat']['playerMicroformatRenderer']['publishDate']
    publishDate = pd.to_datetime(publishDate, format='%Y-%m-%d', errors='ignore')
    
    dataFinal = {
        'videoId': video_id,
        'likes': likes,
        'dislikes': dislikes,
        'talkText': talkText,
        'lengthSeconds': seconds,
        'keywords': keywords,
        'views': views,
        'currentDate': datetime.timestamp(datetime.now()),
        'videoDescription': videoDescription,
        'publishDate': publishDate
    }
#     display(dataFinal)
    with lock:
        videoDataset.loc[video_id, :] = dataFinal


def getSub(data):
    try:
        urlTranscript = data[3]['playerResponse']['captions']['playerCaptionsTracklistRenderer']['captionTracks'][0]['baseUrl']
        response = requests.get(
            url=urlTranscript+'&tlang=en',
            proxies=get_proxie(),
            headers=headersYouTube,
        ).text
        return response
    except:
        return np.nan

# getVideoData('HLIJkmy3vy8')
# getVideoData('6Zex8K6h_jk')
# getVideoData('agsTBnU8plU')
# getVideoData('nVFc6iWk5b0') #no tiene ingles
# getVideoData('U6M5iX9qWh8')

In [7]:
def process_id(vidId):
    """process a single ID"""
    try:
        getVideoData(vidId)
    except:
        print('ERROR IN', vidId)
    return True

def process_range(id_range, store=None):
    total = len(id_range)
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    curr = 0
    for id in id_range:
        print(str(curr)+' of '+str(total))
        curr += 1
        store[id] = process_id(id)
    return store

from threading import Thread

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    return store

threaded_process_range(
    10,
    targets.values,
)

0 of 6552
0 of 6552
0 of 6552
0 of 6551
0 of 6551
0 of 6551
0 of 6551
0 of 6551
0 of 6551
0 of 6551
1 of 6551
1 of 6552
1 of 6552
1 of 6551
1 of 65511 of 6552

1 of 6551
1 of 6551
1 of 6551
1 of 6551
ERROR IN SlGJAS6lYfw
2 of 6551
ERROR IN nLq20wk3WuI
2 of 6552
ERROR IN MOO3qdWVUcw
2 of 6551
2 of 6551
2 of 6551
2 of 6551
2 of 6551
2 of 6552
2 of 6551
3 of 6552
2 of 6552
3 of 6551
3 of 6551
3 of 6551
3 of 6551
3 of 6551
3 of 6552
4 of 6552
3 of 6551
3 of 6551
3 of 6552
4 of 6551
4 of 6551
4 of 6551
4 of 6551
4 of 6552
5 of 6552
4 of 6551
4 of 6552
5 of 6551
5 of 6551
4 of 6551
5 of 6551
4 of 6551
5 of 6552
5 of 6551
6 of 6552
5 of 6552
6 of 6551
6 of 6551
5 of 6551
5 of 6551
5 of 6551
6 of 6551
6 of 6552
6 of 6551
7 of 6552
6 of 6551
7 of 6551
6 of 6552
7 of 6551
6 of 6551
6 of 6551
7 of 6551
7 of 6552
7 of 6551
8 of 6552
7 of 6551
8 of 6551
7 of 6552
8 of 6551
8 of 6551
7 of 6551
8 of 6552
7 of 6551
9 of 6552
8 of 6551
9 of 6551
8 of 6552
8 of 6551
9 of 6551
8 of 6551
8 of 6551
9 of 65

74 of 6551
73 of 6551
76 of 6551
74 of 6551
72 of 6552
74 of 6551
74 of 6552
75 of 6551
77 of 6551
72 of 6551
73 of 6552
77 of 6551
74 of 6551
75 of 6552
73 of 6551
75 of 6551
78 of 6551
75 of 6552
76 of 6551
78 of 6551
75 of 6551
74 of 6552
75 of 6551
76 of 6551
76 of 6552
79 of 6551
74 of 6551
77 of 6551
76 of 6551
75 of 6552
79 of 6551
76 of 6551
77 of 6551
76 of 6552
77 of 6552
75 of 6551
80 of 6551
78 of 6551
80 of 6551
77 of 6551
76 of 6552
77 of 6552
78 of 6552
81 of 6551
78 of 6551
76 of 6551
78 of 6552
77 of 6551
77 of 6552
81 of 6551
79 of 6551
78 of 6551
79 of 6552
82 of 6551
79 of 6551
79 of 6552
78 of 6552
82 of 6551
77 of 6551
80 of 6551
78 of 6551
80 of 6552
79 of 6551
80 of 6552
83 of 6551
83 of 6551
79 of 6552
78 of 6551
80 of 6551
81 of 6551
81 of 6552
81 of 6552
80 of 6551
79 of 655179 of 6551

84 of 6551
81 of 6551
84 of 6551
80 of 6552
85 of 6551
82 of 6552
81 of 6551
82 of 6551
80 of 6551
85 of 6551
82 of 6551
82 of 6552
80 of 6551
83 of 6552
81 of 6552
ERROR IN W

141 of 6551
148 of 6551
146 of 6551
141 of 6552
144 of 6552
142 of 6551
142 of 6551
142 of 6551
148 of 6551
147 of 6551
144 of 6552
146 of 6551
149 of 6551
142 of 6552
145 of 6552
143 of 6551
143 of 6551
143 of 6551
145 of 6552
149 of 6551
148 of 6551
147 of 6551
150 of 6551
146 of 6552
143 of 6552
144 of 6551
144 of 6551
146 of 6552
144 of 6551
147 of 6552
151 of 6551
149 of 6551
150 of 6551
148 of 6551
144 of 6552
145 of 6551
145 of 6551
147 of 6552
152 of 6551
148 of 6552
145 of 6551
148 of 6552
146 of 6551
150 of 6551
149 of 6551
151 of 6551
145 of 6552
153 of 6551
149 of 6552
146 of 6551
146 of 6551
149 of 6552
147 of 6551
146 of 6552
152 of 6551
150 of 6551
151 of 6551
147 of 6551
154 of 6551
150 of 6552
148 of 6551
153 of 6551
147 of 6551
151 of 6551
152 of 6551
150 of 6552
147 of 6552
155 of 6551
148 of 6551
ERROR IN eyBEIx7fCfY
154 of 6551
149 of 6551
148 of 6551
151 of 6552
152 of 6551
148 of 6552
153 of 6551
151 of 6552
156 of 6551
155 of 6551
149 of 6552
149 of 6551
150 of 

211 of 6552
207 of 6551
218 of 6551
211 of 6551
216 of 6551
212 of 6552
210 of 6551
209 of 6552
212 of 6552
211 of 6551
211 of 6551
208 of 6551
212 of 6551
213 of 6552
211 of 6551
217 of 6551
219 of 6551
212 of 6551
210 of 6552
212 of 6551
209 of 6551
213 of 6551
213 of 6552
214 of 6552
212 of 6551
220 of 6551
213 of 6551
213 of 6551
211 of 6552
210 of 6551
214 of 6551
214 of 6552
218 of 6551
215 of 6552
214 of 6551
214 of 6551
213 of 6551
211 of 6551
212 of 6552
221 of 6551
219 of 6551
215 of 6552
215 of 6551
216 of 6552
215 of 6551
212 of 6551
213 of 6552
214 of 6551
222 of 6551
217 of 6552
215 of 6551
220 of 6551
ERROR IN qN5tU3u-c54
221 of 6551
216 of 6552
216 of 6551
216 of 6551
213 of 6551
215 of 6551
223 of 6551
214 of 6552
216 of 6551
ERROR IN218 of 6552
 JdLRw0ErL-c
222 of 6551
217 of 6551
214 of 6551
216 of 6551
217 of 6551
217 of 6552
224 of 6551
217 of 6551
215 of 6552
223 of 6551
219 of 6552
215 of 6551
218 of 6552
218 of 6551
218 of 6551
225 of 6551
216 of 6552
224 of 655

277 of 6552
277 of 6551
281 of 6552
283 of 6551
277 of 6552
285 of 6551
277 of 6551
278 of 6552
278 of 6551
280 of 6551
284 of 6551
278 of 6551
282 of 6552
278 of 6551
279 of 6552
286 of 6551
278 of 6552
278 of 6551
279 of 6551
281 of 6551
285 of 6551
279 of 6551
279 of 6551
280 of 6552
283 of 6552
279 of 6552
282 of 6551
287 of 6551
279 of 6551
280 of 6551
280 of 6551
286 of 6551
284 of 6552
280 of 6551
281 of 6552
280 of 6552283 of 6551

288 of 6551
280 of 6551
281 of 6551
287 of 6551
285 of 6552
281 of 6551
282 of 6552
281 of 6552
281 of 6551
282 of 6551
289 of 6551
281 of 6551
284 of 6551
286 of 6552
288 of 6551
282 of 6551
290 of 6551
283 of 6551
283 of 6552
282 of 6552
282 of 6551
287 of 6552
285 of 6551
282 of 6551
291 of 6551
289 of 6551
283 of 6551
284 of 6552
283 of 6551
288 of 6552
283 of 6552
ERROR IN u-Mqk_q_gN4
283 of 6551
284 of 6551
292 of 6551
286 of 6551
285 of 6552
290 of 6551
284 of 6551
289 of 6552
284 of 6551
285 of 6551
284 of 6552
284 of 6551
293 of 6551
287 of 

...

6297 of 6551
6316 of 6552
6301 of 6551
6325 of 6551
6358 of 6552
6333 of 6551
6308 of 6551
6296 of 6551
6317 of 6552
6311 of 6551
6298 of 6551
6318 of 6552
6359 of 6552
6302 of 6551
6326 of 6551
6309 of 6551
6297 of 6551
6318 of 6552
6334 of 6551
6312 of 6551
6299 of 6551
6319 of 6552
6360 of 6552
6327 of 6551
6303 of 6551
6298 of 6551
6319 of 6552
6335 of 6551
6300 of 6551
6320 of 6552
6310 of 6551
6313 of 6551
6304 of 6551
6328 of 6551
6361 of 6552
6299 of 6551
6320 of 6552
6336 of 6551
6311 of 65516314 of 6551

6301 of 6551
6305 of 6551
6321 of 6552
6329 of 6551
6321 of 6552
6362 of 6552
6337 of 6551
6300 of 6551
6312 of 6551
6302 of 6551
6306 of 6551
6315 of 6551
6322 of 6552
6330 of 6551
6301 of 6551
6322 of 6552
6363 of 6552
6338 of 6551
6313 of 6551
6303 of 6551
6302 of 6551
6316 of 6551
6331 of 6551
6307 of 6551
6323 of 6552
6364 of 6552
6323 of 6552
6339 of 6551
6317 of 6551
6314 of 6551
6304 of 6551
6308 of 6551
6324 of 6552
6332 of 6551
6303 of 6551
6324 of 6552
6365 of 6552

6365 of 6551
6360 of 6551
6419 of 6552
6387 of 6551
6381 of 6552
6371 of 6551
6361 of 6551
6366 of 6551
6397 of 6551
6359 of 6551
6372 of 6551
6383 of 6552
6420 of 6552
6388 of 6551
6372 of 6551
6382 of 6552
6398 of 6551
6421 of 6552
6360 of 6551
6362 of 6551
6367 of 6551
6384 of 6552
6373 of 6551
6373 of 6551
6383 of 6552
6389 of 6551
6385 of 6552
6399 of 6551
6368 of 6551
6422 of 6552
6361 of 6551
6374 of 6551
6390 of 6551
6363 of 6551
6374 of 6551
6384 of 6552
6386 of 6552
6400 of 6551
6391 of 6551
6362 of 6551
6369 of 6551
6375 of 6551
6364 of 6551
6423 of 6552
6375 of 6551
6387 of 6552
6385 of 6552
6392 of 6551
6401 of 6551
6363 of 6551
6370 of 6551
6424 of 6552
6365 of 6551
6388 of 6552
6376 of 6551
6376 of 6551
6402 of 6551
6393 of 6551
6386 of 6552
6371 of 6551
6364 of 6551
6425 of 6552
6366 of 6551
6389 of 6552
6377 of 6551
6377 of 6551
6403 of 6551
6365 of 6551
6394 of 6551
6387 of 6552
6372 of 6551
6390 of 6552
6426 of 6552
6367 of 6551
6378 of 6551
6404 of 6551
6366 of 6551

6423 of 6551
6432 of 6551
6449 of 6551
6423 of 6551
6435 of 6551
6460 of 6551
6444 of 6552
6447 of 6552
6483 of 6552
6433 of 6551
6450 of 6551
6427 of 6551
6424 of 6551
6424 of 6551
6461 of 6551
6445 of 6552
6436 of 6551
6448 of 6552
6434 of 6551
6484 of 6552
6451 of 6551
6425 of 6551
6446 of 6552
6425 of 6551
6428 of 6551
6462 of 6551
6485 of 6552
6435 of 6551
6437 of 6551
6449 of 6552
6426 of 6551
6452 of 6551
6426 of 6551
6429 of 6551
6447 of 6552
6436 of 6551
6486 of 6552
6450 of 6552
6438 of 6551
6463 of 6551
6453 of 6551
6427 of 6551
6448 of 6552
6427 of 6551
6451 of 6552
6437 of 6551
6454 of 6551
6439 of 6551
6430 of 6551
6464 of 6551
6428 of 6551
6428 of 6551
6487 of 6552
6449 of 6552
6452 of 6552
6440 of 6551
6438 of 6551
6429 of 6551
6431 of 6551
6455 of 6551
6488 of 6552
6465 of 6551
6450 of 6552
6429 of 6551
6453 of 6552
6441 of 6551
6439 of 6551
6430 of 6551
6432 of 6551
6454 of 6552
6451 of 6552
6466 of 6551
6489 of 6552
6456 of 6551
6442 of 65516430 of 6551

6440 of 6551

6511 of 6551
6507 of 6552
6489 of 6551
6544 of 6552
6496 of 6551
6486 of 6551
6498 of 6551
6512 of 6552
6522 of 6551
6489 of 6551
6512 of 6551
6508 of 6552
6545 of 6552
6490 of 6551
6497 of 6551
6499 of 6551
6487 of 6551
6513 of 6552
6523 of 6551
6490 of 6551
6491 of 6551
6498 of 6551
6509 of 6552
6546 of 6552
6513 of 6551
6514 of 6552
6500 of 6551
6524 of 6551
6491 of 6551
6488 of 6551
6510 of 6552
6499 of 6551
6492 of 6551
6501 of 6551
6514 of 6551
6515 of 6552
6547 of 6552
6525 of 6551
6492 of 6551
6489 of 6551
6511 of 6552
6516 of 6552
6493 of 6551
6500 of 6551
6515 of 6551
6548 of 6552
6502 of 6551
6526 of 6551
6493 of 6551
6490 of 6551
6512 of 6552
6517 of 6552
6501 of 6551
6494 of 6551
6516 of 6551
6494 of 6551
6549 of 6552
6503 of 6551
6527 of 6551
6491 of 6551
6513 of 6552
6495 of 6551
6502 of 6551
6518 of 6552
6517 of 6551
6495 of 6551
6504 of 6551
6492 of 6551
6514 of 6552
6528 of 6551
6550 of 6552
6496 of 6551
6496 of 6551
6518 of 6551
6503 of 6551
6519 of 6552
6493 of 6551

{'6ZsIms_jAQ0': True,
 'u2eRC-IXkgQ': True,
 'nqSXwWUwpRM': True,
 'QKP-8lmOdmA': True,
 'sXjbTsEoIEE': True,
 'Uq1cdT4MBDc': True,
 'tji-T-IUy94': True,
 '1_FN1ti3r74': True,
 '3SfhBGp_0-Y': True,
 'YY6Bp8Ag8fs': True,
 'SlGJAS6lYfw': True,
 'nLq20wk3WuI': True,
 'MOO3qdWVUcw': True,
 'XXCAzK_mdhc': True,
 'lvTcjPhIP9k': True,
 '9Yh4ijYlE_M': True,
 'cMa8LKQZ8Yw': True,
 'oWOeGiE3Cg4': True,
 'xey0AQ7ASRI': True,
 'cV9wA3JntWw': True,
 '-1KxMC96EAQ': True,
 'vlv7r6uFJP8': True,
 'peV3G58pim4': True,
 '39g_NpToVoY': True,
 'J-gGakOQ9JQ': True,
 'VX3Z7IXIF-I': True,
 'zxHI84Y770o': True,
 'uNXBTsiGGtk': True,
 'LmyRojsJAiA': True,
 'nw5ZETP-fN4': True,
 '9AM1ywuS1IE': True,
 'Ov8wX_TvJmY': True,
 'u19JGixFlpY': True,
 'RduJL0SA09A': True,
 'skQrwXnqTLc': True,
 'ij5OusOeyzM': True,
 '4bbwEu8kuBY': True,
 '6a2CL7GcVy0': True,
 '3s3txLqg3mo': True,
 '-xUDn4FxUFE': True,
 'DyTIqxRwY84': True,
 'SPQF4NVwWvQ': True,
 'Abh56GgwitA': True,
 'DeezZNO6oOc': True,
 'OJ-L41R1oj4': True,
 '0W70y3Bi

In [8]:
print('Charlas:', len(videoDataset), 'of:', len(rawVideos))
videoDataset.head()

Charlas: 96131 of: 96728


,videoId,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,publishDate
qNvKWhzdpW4,qNvKWhzdpW4,0,0,1001,"[TEDxTalks, English, Life, Anthropology, Compl...",222,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Dr. Kyle William Bishop is a Cedar City native...,1.57697e+09,2019-05-15 00:00:00
huOxk-UhOz0,huOxk-UhOz0,468,4,1161,"[TEDxTalks, English, United States, Life, Fail...",26243,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","With an honest and vulnerable approach, Joey H...",1.57697e+09,2016-05-17 00:00:00
J0_6b-AJdWU,J0_6b-AJdWU,21,1,764,"[TEDxTalks, English, Health, Communication, Cu...",1085,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",How limited and limiting do our lives become w...,1.57697e+09,2018-05-22 00:00:00
QjVNcAAf7pA,QjVNcAAf7pA,150,72,927,"[TEDxTalks, English, Social Science, Communica...",12004,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",America has a crisis of conspiracy theory beli...,1.57697e+09,2019-05-16 00:00:00
kP8wJvBXHgs,kP8wJvBXHgs,290,20,685,"[TEDxTalks, English, Life, Change, Youth]",40182,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Guro Heggholmen (11) has a special interest in...,1.57697e+09,2017-10-17 00:00:00


In [9]:
videoDataset.to_pickle('../datasets/rawVideosContent.pkl')